In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [88]:
def scrapGroupPhase(page):
    %%time
    df = pd.DataFrame(columns=['Round','Teams','Score','Bet_Home','Bet_Drawn','Bet_Away','Game_Date','Game_Details','Fase'])
    div = page_content.find_all('ul', attrs = {'class' : 'list-tabs list-tabs--secondary'})
    th_position = 0
    td_position = 0
    data = []
    for content in page:
        for li in content.find_all('li'):
            fase = li.text
            link = li.select('a')[0].attrs['href']
            # fetch the content from url
            page_response = requests.get(page_link+link, timeout=5)
            # parse html
            page_content = BeautifulSoup(page_response.content, "html.parser")
            table = page_content.find_all('table')[0]
            for row in table.find_all(['td', 'th']):
                if row.name == 'th' and th_position == 0:
                    rodada = row.text
                    data = [rodada]
                    th_position = 1
                elif row.name == 'td':
                    if td_position == 0:
                        data.append(row.text)
                        for a in row.find_all('a', href=True):
                            link = a['href']
                    elif 2 <= td_position <= 4:
                        try:
                            data.append(row.select('span')[2].attrs['data-odd'])
                        except:
                            try: 
                                data.append(row.attrs['data-odd'])
                            except:
                                data.append(row.text)                    
                    else:
                        data.append(row.text)
                    td_position+=1       
                    if td_position == 6:
                        data.append(link)
                        data.append(fase)
                        df.loc[len(df)] = data
                        data = [rodada]
                        th_position = 0
                        td_position = 0
    return df

In [89]:
def scrapChampionship(page):
    %%time
    df = pd.DataFrame(columns=['Round','Teams','Score','Bet_Home','Bet_Drawn','Bet_Away','Game_Date','Game_Details'])
    th_position = 0
    td_position = 0
    data = []
    table = page_content.find_all('table')[0]
    for row in table.find_all(['td', 'th']):
        if row.name == 'th' and th_position == 0:
            rodada = row.text
            data = [rodada]
            th_position = 1
        elif row.name == 'td':
            if td_position == 0:
                data.append(row.text)
                for a in row.find_all('a', href=True):
                    link = a['href']
            elif 2 <= td_position <= 4:
                try:
                    data.append(row.select('span')[2].attrs['data-odd'])
                except:
                    try: 
                        data.append(row.attrs['data-odd'])
                    except:
                        data.append(row.text)                    
            else:
                data.append(row.text)
            td_position+=1       
            if td_position == 6:
                data.append(link)
                df.loc[len(df)] = data
                data = [rodada]
                th_position = 0
                td_position = 0
    return df

In [81]:
page_link ='http://www.betexplorer.com/soccer/brazil/serie-a-2017/results/'
# fetch the content from url
page_response = requests.get(page_link, timeout=5)
# parse html
page_content = BeautifulSoup(page_response.content, "html.parser")

In [90]:
%%time
div = page_content.find_all('ul', attrs = {'class' : 'list-tabs list-tabs--secondary'})
if len(div) > 0:
    df = scrapGroupPhase(div)
else:
    df = scrapChampionship(div)

Wall time: 0 ns
Wall time: 4.28 s


In [91]:
df

Round                         Teams Score Bet_Home Bet_Drawn  \
0    38. Round      Atletico GO - Fluminense   1:1     2.35      3.35   
1    38. Round          Atletico-MG - Gremio   4:3     1.24      5.76   
2    38. Round       Atletico-PR - Palmeiras   3:0     2.77      3.32   
3    38. Round        Botafogo RJ - Cruzeiro   2:2     1.75      3.46   
4    38. Round     Chapecoense-SC - Coritiba   2:1     2.10      3.36   
5    38. Round                 Santos - Avai   1:1     1.97      3.54   
6    38. Round             Sao Paulo - Bahia   1:1     1.66      3.74   
7    38. Round    Sport Recife - Corinthians   1:0     1.38      4.69   
8    38. Round           Vasco - Ponte Preta   2:1     1.33      4.73   
9    38. Round         Vitoria - Flamengo RJ   1:2     2.12      3.41   
10   37. Round       Palmeiras - Botafogo RJ   2:0     1.72      3.63   
11   37. Round            Avai - Atletico-PR   1:0     2.57      3.18   
12   37. Round        Bahia - Chapecoense-SC   0:1     1.68      3.72   
13   37. Round     Corinthians - Atletico-MG   2:2     2.17      3.31   
14   37. Round          Coritiba - Sao Paulo   1:2     1.93      3.35   
15   37. Round              Cruzeiro - Vasco   0:1     1.89      3.45   
16   37. Round          Flamengo RJ - Santos   1:2     1.62      3.62   
17   37. Round          Gremio - Atletico GO   1:1     1.72      3.49   
18   37. Round         Ponte Preta - Vitoria   2:3     2.12      3.29   
19   37. Round     Fluminense - Sport Recife   1:2     2.15      3.36   
20   36. Round              Avai - Palmeiras   2:1     3.61      3.45   
21   36. Round      Fluminense - Ponte Preta   2:0     1.81      3.40   
22   36. Round  Atletico GO - Chapecoense-SC   1:1     2.42      3.23   
23   36. Round        Atletico-MG - Coritiba   3:0     1.55      4.02   
24   36. Round           Atletico-PR - Vasco   3:1     2.01      3.26   
25   36. Round     Flamengo RJ - Corinthians   3:0     1.64      3.66   
26   36. Round               Santos - Gremio   1:0     1.49      4.04   
27   36. Round       Sao Paulo - Botafogo RJ   0:0     2.07      3.26   
28   36. Round          Sport Recife - Bahia   1:0     1.96      3.51   
29   36. Round            Vitoria - Cruzeiro   1:1     1.96      3.43   
..         ...                           ...   ...      ...       ...   
350   3. Round         Chapecoense-SC - Avai   2:0     1.58      3.58   
351   3. Round           Botafogo RJ - Bahia   1:0     1.96      3.24   
352   3. Round     Atletico GO - Corinthians   0:1     4.04      3.21   
353   3. Round     Atletico-MG - Ponte Preta   2:2     1.50      4.07   
354   3. Round     Atletico-PR - Flamengo RJ   1:1     2.84      3.10   
355   3. Round             Santos - Cruzeiro   0:1     1.95      3.27   
356   3. Round         Sao Paulo - Palmeiras   2:0     2.44      3.08   
357   3. Round         Sport Recife - Gremio   4:3     1.87      3.27   
358   3. Round            Vitoria - Coritiba   0:1     2.12      3.16   
359   3. Round            Vasco - Fluminense   3:2     2.61      3.19   
360   2. Round              Sao Paulo - Avai   2:0     1.32      4.71   
361   2. Round       Sport Recife - Cruzeiro   1:1     2.59      3.11   
362   2. Round     Atletico GO - Flamengo RJ   0:3     3.95      3.32   
363   2. Round      Atletico-MG - Fluminense   1:2     1.63      3.89   
364   2. Round          Atletico-PR - Gremio   0:2     2.52      3.02   
365   2. Round     Botafogo RJ - Ponte Preta   2:0     2.05      3.22   
366   2. Round    Chapecoense-SC - Palmeiras   1:0     3.00      3.16   
367   2. Round                 Vasco - Bahia   2:1     1.81      3.46   
368   2. Round         Vitoria - Corinthians   0:1     3.10      3.10   
369   2. Round             Santos - Coritiba   1:0     1.58      3.85   
370   1. Round        Coritiba - Atletico GO   4:1     1.59      3.58   
371   1. Round          Gremio - Botafogo RJ   2:0     1.69      3.44   
372   1. Round                Avai - Vitoria   0:0     2.31      3.